- Adopt standard variable convention for op, op_str, etc.
    - matrixoperators are given name mat_op, singletrace are given st_op
    - op string names 'X', 'P' are op_str
    - operator tuples are op
- how to handle 'empty' operator SingleTraceOperator(data={():0})

In [1]:
import numpy as np
import sympy as sp
from typing import Union, Self
from numbers import Number
from itertools import chain
from bmn.algebra import MatrixOperator, SingleTraceOperator, MatrixSystem
from itertools import product
from scipy.sparse import coo_matrix

In [26]:
np.asarray([
    [1, 2, 3],
    [4, 5, 6],
]).shape

(2, 3)

In [27]:
def create_sparse_matrix_from_dict(index_value_dict: dict[tuple[int, int], Number], matrix_shape: tuple[int, int]) -> coo_matrix:
    """
    Create a sparse COO-formatted matrix from an index-value dictionary.

    Parameters
    ----------
    index_value_dict : dict[tuple[int, int], Number]
        The index-value dictionary, formatted as in:

        index_value_dict = {
            (0, 1): 4,
            (1, 2): 7,
            (2, 0): 5,
            (3, 3): 1
        }

    matrix_shape: tuple[int, int]
        The matrix shape, required because the matrix is sparse.

    Returns
    -------
    coo_matrix
        The sparse COO matrix.
    """

    # prepare the data
    row_indices = []
    column_indices = []
    data_values = []

    for (row, col), value in index_value_dict.items():
        row_indices.append(row)
        column_indices.append(col)
        data_values.append(value)

    # convert lists to numpy arrays
    row_indices = np.array(row_indices)
    column_indices = np.array(column_indices)
    data_values = np.array(data_values)

    print('(num_rows, num_cols) = ', (num_rows, num_cols))
    # create the sparse matrix
    sparse_matrix = coo_matrix((data_values, (row_indices, column_indices)), shape=(num_rows, num_cols))

    print(sparse_matrix.shape)

    return sparse_matrix

## One Matrix Model

In [28]:
class BootstrapSystem():
    """
    _summary_
    """
    def __init__(
            self,
            matrix_system: MatrixSystem,
            hamiltonian: SingleTraceOperator,
            gauge: MatrixOperator,
            half_max_degree: int
            ):
        self.matrix_system = matrix_system
        self.hamiltonian = hamiltonian
        self.gauge = gauge
        self.half_max_degree = half_max_degree
        self.operator_list = self.generate_operators(2*half_max_degree)
        self.operator_dict = {op: idx for idx, op in enumerate(self.operator_list)}
        if 2*self.half_max_degree < self.hamiltonian.max_degree:
            raise ValueError("2 * half_max_degree must be >= max degree of Hamiltonian.")

    def generate_operators(self, max_degree: int) -> list[str]:
        """
        Generate the list of operators used in the bootstrap, i.e.
            I, X, P, XX, XP, PX, PP, ...,
        up to and including strings of max_degree degree.

        Parameters
        ----------
        max_degree : int
            Maximum degree of operators to consider.

        Returns
        -------
        list[str]
            A list of the operators.
        """
        operators = {deg: [x for x in product(self.matrix_system.operator_basis, repeat=deg)] for deg in range(0, max_degree+1)}
        self.psd_matrix_dim = sum(len(value) for degree, value in operators.items() if degree <= self.half_max_degree)
        return [x for xs in operators.values() for x in xs] # flatten

    def generate_hamiltonian_constraints(self) -> list[SingleTraceOperator]:
        """
        Generate the Hamiltonian constraints <[H,O]>=0 for O single trace.

        Returns
        -------
        list[SingleTraceOperator]
            The list of constraint terms.
        """
        constraints = []
        for op in self.operator_list:
             constraints.append(self.matrix_system.single_trace_commutator(
                 st_operator1=self.hamiltonian,
                 st_operator2=SingleTraceOperator(data={op: 1})
                 ))
        return self.clean_constraints(constraints)

    def generate_gauge_constraints(self) -> list[SingleTraceOperator]:
        """
        Generate the Gauge constraints <tr(G O)>=0 for O a general matrix operator.
        Because G has max degree 2, this will create constraints involving terms
        with degree max_degree + 2. These will be discarded.

        Returns
        -------
        list[SingleTraceOperator]
            The list of constraint terms.
        """
        constraints = []
        for op in self.operator_list:
             constraints.append(
                 (self.gauge * MatrixOperator(data={op: 1})).trace()
                 )
        return self.clean_constraints(constraints)

    def generate_reality_constraints(self) -> list[SingleTraceOperator]:
        """
        Generate single trace constraints imposed by reality,
            <O^dagger> = <O>

        Returns
        -------
        list[SingleTraceOperator]
            The list of constraint terms.
        """
        constraints = []
        for op in self.operator_list:
             st_operator = SingleTraceOperator(data={op: 1})
             st_operator_dagger = SingleTraceOperator(data={op: 1}).hermitian_conjugate()
             if len(st_operator - st_operator_dagger) > 0:
                 constraints.append(st_operator - st_operator_dagger)
        return self.clean_constraints(constraints)

    def generate_cyclic_constraints(self) -> list[SingleTraceOperator]:
        # TODO it might be nice to implement this using a DoubleTraceOperator class
        constraints = {}
        for idx, op in enumerate(self.operator_list):
            if len(op) > 1:
                # the LHS corresponds to single trace operators
                # note that in Eq S37, S38 their RHS contains single trace operators (k=1, k=r) in their notation
                eq_lhs = SingleTraceOperator(data={op: 1}) - SingleTraceOperator(data={tuple(op[1:]) + tuple(op[0]): 1})
                #eq_lhs += self.matrix_system.commutation_rules[(op_str[0], op_str[1])] * SingleTraceOperator(data={tuple(op_str[2:]): 1})
                #eq_lhs += self.matrix_system.commutation_rules[(op_str[0], op_str[len(op_str)-1])] * SingleTraceOperator(data={tuple(op_str[1:len(op_str)-1]): 1})

                # rhe RHS corresponds to double trace operators
                eq_rhs = []
                for k in range(1, len(op)):
                    commutator = self.matrix_system.commutation_rules[(op[0], op[k])]
                    eq_rhs.append([commutator, SingleTraceOperator(data={tuple(op[1:k]): 1}), SingleTraceOperator(data={tuple(op[k+1:]): 1})])
                constraints[idx] = {'lhs':eq_lhs, 'rhs':eq_rhs}
        return constraints #self.clean_constraints(constraints)

    def build_linear_constraints(self, return_matrix: bool=True):
        """
        _summary_
        1. accumulate each constraint and map it to a vector
        2. reduce system of equations
        3. what about odd number of matrices?
        4. return sparse matrix
        """
        empty_operator = SingleTraceOperator(data={():0})
        constraints = []

        # Hamiltonian constraints
        for st_operator in self.generate_hamiltonian_constraints():
            if st_operator != empty_operator:
                constraints.append({op: coeff for op, coeff in st_operator})

        # gauge constraints
        for st_operator in self.generate_gauge_constraints():
            if st_operator != empty_operator:
                constraints.append({op: coeff for op, coeff in st_operator})

        # reality constraints
        for st_operator in self.generate_reality_constraints():
            if st_operator != empty_operator:
                constraints.append({op: coeff for op, coeff in st_operator})

        # optionally return the constraints in a human-readable form
        if not return_matrix:
            return constraints

        # build the index-value dict
        index_value_dict = {}
        for idx_constraint, constraint_dict in enumerate(constraints):
            print(idx_constraint)
            for op, coeff in constraint_dict.items():
                index_value_dict[(idx_constraint, self.operator_dict[op])] = coeff

        return create_sparse_matrix_from_dict(index_value_dict=index_value_dict, matrix_shape=(len(constraints), len(self.operator_list)))

    def clean_constraints(self, constraints: list[SingleTraceOperator]) -> list[SingleTraceOperator]:
        """
        Remove constraints that involve operators outside the basis set.

        Parameters
        ----------
        constraints : list[SingleTraceOperator]
            The single trace constraints.

        Returns
        -------
        list[SingleTraceOperator]
            The cleaned constraints.
        """
        cleaned_constraints = []
        for st_operator in constraints:
            if all([op in self.operator_list for op in st_operator.data]):
                cleaned_constraints.append(st_operator)
        return cleaned_constraints

In [29]:
matrix_system = MatrixSystem(
    operator_basis=['X', 'P'],
    commutation_rules_concise = {
        ('P', 'X'): -1j,
    }
)

# scale variables as P = sqrt(N) P', X = sqrt(N) X'
hamiltonian = SingleTraceOperator(
        data={("P", "P"): 1, ("X", "X"): 1, ("X", "X", "X", "X"): 7}
    )

# <tr G O > = 0 might need to be applied only for O with deg <= L-2
gauge = MatrixOperator(data={('X', 'P'): 1j, ('P', 'X'): -1j, ():1})

bootstrap = BootstrapSystem(
    matrix_system=matrix_system,
    hamiltonian=hamiltonian,
    gauge=gauge,
    half_max_degree=2
)
#bootstrap.operator_list[:bootstrap.psd_matrix_dim]
bootstrap.matrix_system.commutation_rules

{('X', 'X'): 0, ('P', 'X'): (-0-1j), ('X', 'P'): 1j, ('P', 'P'): 0}

In [30]:
matrix_system.operator_basis

['X', 'P']

In [31]:
bootstrap.operator_dict

{(): 0,
 ('X',): 1,
 ('P',): 2,
 ('X', 'X'): 3,
 ('X', 'P'): 4,
 ('P', 'X'): 5,
 ('P', 'P'): 6,
 ('X', 'X', 'X'): 7,
 ('X', 'X', 'P'): 8,
 ('X', 'P', 'X'): 9,
 ('X', 'P', 'P'): 10,
 ('P', 'X', 'X'): 11,
 ('P', 'X', 'P'): 12,
 ('P', 'P', 'X'): 13,
 ('P', 'P', 'P'): 14,
 ('X', 'X', 'X', 'X'): 15,
 ('X', 'X', 'X', 'P'): 16,
 ('X', 'X', 'P', 'X'): 17,
 ('X', 'X', 'P', 'P'): 18,
 ('X', 'P', 'X', 'X'): 19,
 ('X', 'P', 'X', 'P'): 20,
 ('X', 'P', 'P', 'X'): 21,
 ('X', 'P', 'P', 'P'): 22,
 ('P', 'X', 'X', 'X'): 23,
 ('P', 'X', 'X', 'P'): 24,
 ('P', 'X', 'P', 'X'): 25,
 ('P', 'X', 'P', 'P'): 26,
 ('P', 'P', 'X', 'X'): 27,
 ('P', 'P', 'X', 'P'): 28,
 ('P', 'P', 'P', 'X'): 29,
 ('P', 'P', 'P', 'P'): 30}

In [32]:
len(bootstrap.build_linear_constraints(return_matrix=False)), len(bootstrap.operator_dict)

(33, 31)

In [33]:
bootstrap.build_linear_constraints(return_matrix=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
(num_rows, num_cols) =  (79, 31)
(79, 31)


<79x31 sparse matrix of type '<class 'numpy.complex128'>'
	with 79 stored elements in COOrdinate format>

In [ ]:
bootstrap.operator_list[0], bootstrap.generate_hamiltonian_constraints()[0]

In [ ]:
bootstrap.operator_list[1], bootstrap.generate_hamiltonian_constraints()[1]

In [ ]:
bootstrap.operator_list[2], bootstrap.generate_hamiltonian_constraints()[2]

In [ ]:
bootstrap.operator_list[3], 1j * 1/2 * bootstrap.generate_hamiltonian_constraints()[3]

In [ ]:
bootstrap.generate_hamiltonian_constraints()[3], bootstrap.generate_hamiltonian_constraints()[3].hermitian_conjugate()

In [ ]:
bootstrap.operator_list[0], -1j * bootstrap.generate_gauge_constraints()[0]

In [ ]:
# this reproduces Eq 14 in the main text (promote this to a unit test)
idx = bootstrap.operator_list.index(('X', 'P', 'P', 'P'),)
bootstrap.generate_cyclic_constraints()[idx]

## OLD

In [ ]:
matrix_system = MatrixSystem(
    operator_basis=['X0', 'P1'],
    commutation_rules_concise = {
        ('P0', 'X0'): -1j,
        ('P1', 'X1'): -1j,
        ('P2', 'X2'): -1j,
    }
)
len(matrix_system.commutation_rules)